- #### Transformaciones
    - ##### join
    - ##### UDF

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("sesion_1") \
        .master("local[*]") \
        .getOrCreate()

In [2]:
def read_csv(path):
    return spark.read\
        .option("header","true")\
        .option("delimiter",",")\
        .option("inferSchema","false")\
        .csv(path)

base_path = "../resources/data/csv/"
clients_df = read_csv(base_path + "clients.csv")
contracts_df = read_csv(base_path + "contracts.csv")
products_df = read_csv(base_path + "products.csv")

clients_df.show()
contracts_df.show()
products_df.show()

+----------+-------+----+---------+----------+-----+
|cod_client| nombre|edad|provincia|cod_postal|  vip|
+----------+-------+----+---------+----------+-----+
|     00001|  Julen|  31| Valencia|     23700|false|
|     00002| Javier|  58|     Jaén|     08728| true|
|     00003| Carlos|  48|  Sevilla|     28757| true|
|     00004|  Maria|  38|   Gerona|     18393| true|
|     00005|Ernesto|  22|  Cáceres|     13950|false|
|     00006|  Luisa|  43|   Murcia|     23940| true|
|     00007|   Raul|  51|    Álava|     09030|false|
+----------+-------+----+---------+----------+-----+

+-------+-----------+------------+----------+------+
|cod_iuc|cod_titular|cod_producto|  fec_alta|activo|
+-------+-----------+------------+----------+------+
|  30000|      00006|         100|2012-05-01|  true|
|  30001|      00006|         200|2014-05-01|  true|
|  30002|      00006|         300|2006-02-01| false|
|  30003|      00006|         150|2012-05-01|  true|
|  30002|      00005|         300|2012-05-01|

In [3]:
# Joins

# inner -> Mantiene información de ambas tablas (columnas) para los registros (filas) coincidentes
# outer -> Mantiene información de ambas tablas (columnas y filas) para los registros coincidentes y no-coincidentes
# left -> Mantiene columnas de ambas tablas y filas únicamente de la tabla izquierda, elimina filas no coincidentes de la tabla derecha
# right -> Mantiene columnas de ambas tablas y filas únicamente de la tabla derecha, elimina filas no coincidentes de la tabla izquierda
# left_semi -> Mantiene filas y columnas únicamente de la tabla izquierda para los registros coincidentes
# left_anti -> Mantiene filas y columnas únicamente de la tabla izquierda para los registros no-coincidentes

# cross

In [20]:
import pyspark.sql.functions as f

clients_tmp_df = clients_df.filter((f.col("edad") >= 40) & (f.col("edad") <= 50))
contracts_tmp_df = contracts_df.filter(f.col("activo") == "false")\
    .withColumnRenamed("cod_titular", "cod_client")

clients_tmp_df.show()
contracts_tmp_df.show()

# clients_tmp_df.crossJoin(contracts_tmp_df).show() ## WARNING

typw_join = "full"   # inner, outer, left, right, left_semi, left_anti

join_df = clients_tmp_df.join(contracts_tmp_df, ["cod_client"], typw_join)
join_df.show()

+----------+------+----+---------+----------+----+
|cod_client|nombre|edad|provincia|cod_postal| vip|
+----------+------+----+---------+----------+----+
|     00003|Carlos|  48|  Sevilla|     28757|true|
|     00006| Luisa|  43|   Murcia|     23940|true|
+----------+------+----+---------+----------+----+

+-------+----------+------------+----------+------+
|cod_iuc|cod_client|cod_producto|  fec_alta|activo|
+-------+----------+------------+----------+------+
|  30002|     00006|         300|2006-02-01| false|
|  30004|     00006|         400|2012-05-01| false|
|  30006|     00006|         600|2012-05-01| false|
|  30007|     00006|         700|2014-02-01| false|
|  30002|     00007|         300|2001-04-11| false|
|  30004|     00002|         400|2008-02-01| false|
|  30010|     00007|        1000|2014-08-01| false|
+-------+----------+------------+----------+------+

+----------+------+----+---------+----------+----+-------+----------+------------+----------+------+
|cod_client|nombre|

In [31]:
# UDF - User Defined Function - WARNING
import pyspark.sql.types as t

def upperCase(value):
    if value is None:
        return ""
    else:
        return value.upper()

def len_concat(item_1, item_2):
    if item_1 is None:
        item_1 = ""
    if item_2 is None:
        item_2 = ""
    return len(item_1 + item_2)

upper_udf = f.udf(upperCase, t.StringType())

len_concat_udf = f.udf(len_concat, t.LongType())

join_df.select(
    *join_df.columns,
    upper_udf(f.col("nombre")).alias("nombre_mayus"),
    len_concat_udf(f.col("nombre"), f.col("provincia")).alias("len_concat")
).show()

+----------+------+----+---------+----------+----+-------+------------+----------+------+------------+----------+
|cod_client|nombre|edad|provincia|cod_postal| vip|cod_iuc|cod_producto|  fec_alta|activo|nombre_mayus|len_concat|
+----------+------+----+---------+----------+----+-------+------------+----------+------+------------+----------+
|     00002|  null|null|     null|      null|null|  30004|         400|2008-02-01| false|            |         0|
|     00003|Carlos|  48|  Sevilla|     28757|true|   null|        null|      null|  null|      CARLOS|        13|
|     00006| Luisa|  43|   Murcia|     23940|true|  30002|         300|2006-02-01| false|       LUISA|        11|
|     00006| Luisa|  43|   Murcia|     23940|true|  30004|         400|2012-05-01| false|       LUISA|        11|
|     00006| Luisa|  43|   Murcia|     23940|true|  30006|         600|2012-05-01| false|       LUISA|        11|
|     00006| Luisa|  43|   Murcia|     23940|true|  30007|         700|2014-02-01| false

In [39]:
from pyspark.sql import Row

tmp_df = spark.createDataFrame([
    Row(1,None),
    Row(2,float("nan")),
    Row(3,3.2),
    Row(4,float("nan"))], ["id", "number"])

In [42]:
tmp_df.show()

def is_null_or_nan(value):
    if (value!=value) | (value is None):
        return True
    else:
        return False

is_null_or_nan_udf = f.udf(is_null_or_nan, t.BooleanType())

tmp_df.select(
    *tmp_df.columns,
    is_null_or_nan_udf(f.col("number")).alias("null_nan")
).show()

+---+------+
| id|number|
+---+------+
|  1|  null|
|  2|   NaN|
|  3|   3.2|
|  4|   NaN|
+---+------+

+---+------+--------+
| id|number|null_nan|
+---+------+--------+
|  1|  null|    true|
|  2|   NaN|    true|
|  3|   3.2|   false|
|  4|   NaN|    true|
+---+------+--------+

